In [1]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
import agrege_troncon as at
import Connexion_Transfert as ct
import matplotlib
import geopandas as gp
import numpy as np
from datetime import datetime
import Connexion_Transfert as ct
from shapely.wkt import loads
import Outils

In [2]:
#importer les odnnées de  base
with ct.ConnexionBdd('local_otv') as c : 
    df = gp.read_postgis("select t.*, v1.cnt nb_intrsct_src, st_astext(v1.the_geom) as src_geom, v2.cnt as nb_intrsct_tgt, st_astext(v2.the_geom) as tgt_geom from public.test_agreg_lineaire t left join public.test_agreg_lineaire_vertices_pgr v1 on t.\"source\"=v1.id left join public.test_agreg_lineaire_vertices_pgr v2  on t.target=v2.id", c.connexionPsy)
nature_2_chaussees=['Route à 2 chaussées','Quasi-autoroute','Autoroute']
#donnees generale du jdd
df.set_index('id_ign', inplace=True)  # passer l'id_ign commme index ; necessaire sinon pb avec geometrie vu comme texte
df2_chaussees=df.loc[df.loc[:,'nature'].isin(nature_2_chaussees)] #isoler les troncon de voies decrits par 2 lignes

In [44]:
toto=at.affecter_troncon_ligne('TRONROUT0000000109674477')

0eme occurence : TRONROUT0000000109674477 à 20:31:42 nb ligne traite : 0, nb ligne differente=0


In [45]:
toto

{'TRONROUT0000000109674477': 0,
 'TRONROUT0000000032841957': 0,
 'TRONROUT0000000130923326': 0,
 'TRONROUT0000000032839277': 0,
 'TRONROUT0000000032839286': 0,
 'TRONROUT0000000032839287': 0,
 'TRONROUT0000000032839282': 0,
 'TRONROUT0000000032837045': 0,
 'TRONROUT0000000032837041': 0,
 'TRONROUT0000000032834970': 0,
 'TRONROUT0000000032839279': 0,
 'TRONROUT0000000032839288': 0,
 'TRONROUT0000000032839278': 0,
 'TRONROUT0000000032837043': 0,
 'TRONROUT0000000032837069': 0,
 'TRONROUT0000000032834969': 0}

In [21]:
a.add(element for element in ['TRONROUT0000000222486973', 'TRONROUT0000000032900975'])

In [34]:
a

{'b', 'c', 'v'}